In [13]:
import csv
import re
from tqdm import tqdm_notebook

def cleanAndWriteStory(story, file):
    storyNoIntro = story[1:]
    # Get rid of linespace
    storyNoIntro.pop(1)

    # get rid of actions and write to file
    for lineID, line in enumerate(storyNoIntro):
        lineStripped = re.sub(r"{.*}", "", line)
        lineStripped = re.sub(r"\[.*\]", "", lineStripped)
        # only keep the narrator's lines (scealextric_paragraphs)
#         lineStripped = re.sub(r"[AB]:.*(N:)", "", lineStripped)
#         lineStripped = re.sub(r"\|.*", "", lineStripped)
#         # Narrator's lines
#         lineStripped = re.sub(r"N:(.*)", "\g<1>.", lineStripped)
        # put every character on a new line (needed for scealextric_linebreaks)
        lineStripped = lineStripped.replace(":", ": ")
        lineStripped = lineStripped.replace("|", ".\n")
        lineStripped = lineStripped.replace("?.", "?")
        if lineID == 0:
            lineStripped = re.sub(r"(.;)(.)", "\g<1> \g<2>", lineStripped)
#             file.write(lineStripped + "\n")
#         else:
#             file.write(lineStripped + " ")
        file.write(lineStripped + "\n")
    file.write("<|endoftext|>\n")
    
# The story file is expected to be tab delimited
fromFile = "storyville"
n = 10000
# A single file may be too big to fit in memory, if so: create smaller files
createSingleFile = False
with open(f"data/{fromFile}.txt", "r", encoding="utf-8") as f:
    stories = csv.reader(f, delimiter="\t")
    
    if createSingleFile:
        with open(f"data/{fromFile}Stripped.txt", "w", encoding="utf-8") as newF:
            for storyID, story in enumerate(stories):
                cleanAndWriteStory(story, newF)
    else:
        i = -1
        for storyID, story in enumerate(tqdm_notebook(stories, desc="Stories", total=504101)):
            if storyID%n == 0:
                i += 1
            # appends the stories to the files. When creating new stories delete the files in the stories folder
            with open(f"data/stories_scealextric_linebreaks/{fromFile}Stripped{i}.txt", "a", encoding="utf-8") as newF:
                cleanAndWriteStory(story, newF)

# A single stripped file ended up being 1.5GB and resulted in a memory error when calculating its BPE
# Hopefully splitting the stories into multiple files does not produce a OOM error.